In [ ]:
import torch
from torch import Tensor
from torch import nn
from torch import optim
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import torchvision as tv
from torchvision import models
import os
import numpy
from tqdm import tqdm

import zipfile
from google.colab import drive

drive.mount('/content/drive')
zipped_test = '/content/drive/MyDrive/для кодов/маффины/test.zip'
zipped_train ='/content/drive/MyDrive/для кодов/маффины/test.zip'
z = zipfile.ZipFile(zipped_test, 'r')
z.extractall(path='/content/dataset/test')
z = zipfile.ZipFile(zipped_train, 'r')
z.extractall(path='/content/dataset/train')
import gc
gc.collect()


DATASET_ROOT_DIR = '/content/dataset/'
BATCH_SIZE = 64
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
device


transforms = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor(),
    tv.transforms.RandomHorizontalFlip(0.5),
])

def is_valid_file(path):
    """Checks if a file is a valid image file and not a hidden macOS file."""
    return not path.startswith('._') and os.path.splitext(path)[-1].lower() in ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp']

dataset = ImageFolder(root=os.path.join(DATASET_ROOT_DIR, 'train'),
                      transform=transforms,
                      is_valid_file=is_valid_file
                      ) # 0 - dogs


train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


train_data = DataLoader(
                  train_dataset,
                  batch_size=BATCH_SIZE,
                  shuffle=True,
                  )
test_data = DataLoader(
                  test_dataset,
                  batch_size=BATCH_SIZE,
                  shuffle=True,
                  )
class CustomResNet(nn.Module):
    def __init__(self):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=False)

        self.ep = 0

        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 2)

    def forward(self, x):
        return self.resnet(x)



model = CustomResNet()
model.to(device)


actual_ep=175
model_path = f'/content/drive/MyDrive/Colab Notebooks/models/resnet_model_x50_{actual_ep}'
if os.path.exists(model_path):
    model = torch.load(model_path)
else:
    print(f"Warning: Model file not found at {model_path}. Starting with a new model.")
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-4)


def save_model_to_drive(model, name):
  pth = os.path.join('/content/drive/MyDrive/Colab Notebooks/models', name)
  torch.save(model, pth)


def get_acc():
  correct = 0
  total = 0

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [69]:
import os
from tqdm import tqdm
from PIL import Image
import torch
from torch import nn, optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader

# Dataset class with filtering
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for root, _, files in os.walk(root_dir):
            if '__MACOSX' in root:
                continue
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.image_paths.append(os.path.join(root, file))
                    label = 0 if 'class0' in os.path.basename(os.path.dirname(os.path.join(root, file))) else 1
                    self.labels.append(label)


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            return None, None

        if self.transform:
            image = self.transform(image)

        return image, label


class CustomResNet(nn.Module):
    def __init__(self):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=False)
        self.ep = 0
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 2)

    def forward(self, x):
        return self.resnet(x)


def save_model_to_drive(model, name):
    pth = os.path.join('/content/drive/MyDrive/Colab Notebooks/models', name)
    torch.save(model, pth)


def get_acc(data_loader, model, device):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for img, label in data_loader:
            img = img.to(device)
            label = label.to(device)
            out = model(img)
            _, predicted = torch.max(out.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()

    return correct / total


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_data = CustomDataset('/content/dataset/train', transform=transform)
test_data = CustomDataset('/content/dataset/test', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

model = CustomResNet()
model.to(device)

actual_ep = 175
model_path = f'/content/drive/MyDrive/Colab Notebooks/models/resnet_model_x50_{actual_ep}'
if os.path.exists(model_path):
    model = torch.load(model_path)
    print(f"Model loaded from {model_path}")
else:
    print(f"Warning: Model file not found at {model_path}. Starting with a new model.")

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 1

for i in range(EPOCHS):
    model.train()
    for img, label in tqdm(train_loader):
        optimizer.zero_grad()
        img = img.to(device)
        label = label.to(device)

        out = model(img)
        loss = loss_func(out, label)
        loss.backward()
        optimizer.step()
    model.ep += 1

    print(f'Epoch {model.ep}, Loss: {loss.item()}')

    if model.ep % 5 == 0:
        acc = get_acc(test_loader, model, device)
        print(f'Accuracy: {acc}')
        save_model_to_drive(model, f'resnet_model_x50_{model.ep}')
        print('Model saved!')

final_acc = get_acc(test_loader, model, device)
print(f'Final accuracy: {final_acc}')


100%|██████████| 37/37 [05:00<00:00,  8.12s/it]


Epoch 1, Loss: 0.004813789390027523
Final accuracy: 1.0


In [64]:
import os
from tqdm import tqdm
from PIL import Image
import torch
from torch import nn, optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
from datetime import datetime

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class CustomResNet(nn.Module):
    def __init__(self):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=False)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 2)

    def forward(self, x):
        return self.resnet(x)


class MyDataset(Dataset):
    def __init__(self, root, transforms=None):
        all_files = os.listdir(root)
        print(f"All files in directory: {all_files}")

        self.images = [
            f for f in all_files
            if os.path.isfile(os.path.join(root, f)) and
            '__MACOSX' not in f and
            any(f.lower().endswith(ext) for ext in ('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.dog', '.muf'))
        ]

        if not self.images:
            raise ValueError(
                f"No valid image files found in directory: {root}. "
                f"Check the directory path and file extensions. Found: {all_files}"
            )

        print(f"Filtered images: {self.images}")

        self.root = root
        self.transforms = transforms

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root, self.images[idx])
        image = Image.open(img_name).convert('RGB')
        if self.transforms:
            image = self.transforms(image)
        return image, img_name

transforms_ = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

model = CustomResNet().to(device)

print(f'CustomResNet parameters: {sum(p.numel() for p in model.parameters()):,}')
resnext_model = models.resnext50_32x4d(pretrained=False)
print(f'ResNext50_32x4d parameters: {sum(p.numel() for p in resnext_model.parameters()):,}')

test_dataset_path = '/content/dataset/test'

if not os.path.exists(test_dataset_path):
    print(f"The directory {test_dataset_path} does not exist.")
else:
    files_in_dir = os.listdir(test_dataset_path)
    print(f"Files in {test_dataset_path}: {files_in_dir}")

    try:
        d = MyDataset(test_dataset_path, transforms_)

        dtldr = DataLoader(d, batch_size=1, shuffle=True)

        for i, (img, img_name) in enumerate(dtldr):
            img = img.to(device)
            out = model(img)
            class_label = torch.argmax(out).item()

            name = 'dog' if class_label == 0 else 'muf'

            base, ext = os.path.splitext(img_name[0])
            new_name = os.path.join(test_dataset_path, f"{name}_{i}{ext}")
            os.rename(img_name[0], new_name)
            print(f"Renamed {img_name[0]} to {new_name}")
    except ValueError as e:
        print(e)



Using device: cpu
CustomResNet parameters: 11,177,538
ResNext50_32x4d parameters: 25,028,904
Files in /content/dataset/test: ['muffin', '__MACOSX', 'chihuahua']
All files in directory: ['muffin', '__MACOSX', 'chihuahua']
No valid image files found in directory: /content/dataset/test. Check the directory path and file extensions. Found: ['muffin', '__MACOSX', 'chihuahua']


In [67]:
import os
from PIL import Image
import pandas as pd
import zipfile

test_zip_path = '/content/drive/MyDrive/для кодов/маффины/test.zip'
new_extracted_folder = '/content/dataset/new_test/'


with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall(new_extracted_folder)

extracted_folder = new_extracted_folder


predictions = []

model.eval()

for image_file in os.listdir(extracted_folder):
    image_path = os.path.join(extracted_folder, image_file)


    if os.path.isfile(image_path):

        with Image.open(image_path) as img:

            img = transforms(img).unsqueeze(0).to(device)


            with torch.no_grad():
                model.eval()
                output = model(img)
                _, predicted = torch.max(output, 1)


            predictions.append({'ID': image_file, 'Label': predicted.item()})
            #0=dogs  1=muffins


predictions_df = pd.DataFrame(predictions)


predictions_df.to_csv('/content/drive/MyDrive/Colab Notebooks/predictions.csv', index=False)


In [ ]:
import csv
import os
from pathlib import Path

output_csv_path = '/content/drive/My Drive/results.csv'

results = ["muffin" if value else "chihuahua" for value in y_pred]

labels = list(valid_files.iterdir())

with open(output_csv_path, "w", newline="") as f:
    fieldnames = ["ID", "Label"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for label, result in zip(labels, results):
        writer.writerow({fieldnames[0]: label.name, fieldnames[1]: result})

print(f"Results saved to {output_csv_path}")

# Новый раздел